# 02: Cross-validation

## p28 binの確認

In [1]:
# stratified-kfold for regression 
import numpy as np 
import pandas as pd 
 
from sklearn import datasets 
from sklearn import model_selection 

In [2]:
# we create a sample dataset with 15000 samples  
# and 100 features and 1 target 
X, y = datasets.make_regression( 
    n_samples=15000, n_features=100, n_targets=1 
) 
 
# create a dataframe out of our numpy arrays 
df = pd.DataFrame( 
    X, 
    columns=[f"f_{i}" for i in range(X.shape[1])] 
) 
df.loc[:, "target"] = y 

In [3]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_91,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target
0,0.140460,0.198421,-0.574314,1.060653,-0.758788,1.317201,0.877023,-0.549061,-0.546976,-2.148835,...,-0.450533,-0.219189,-0.109742,0.592466,-1.906363,0.013963,-0.315500,-0.265236,-0.243782,161.886006
1,-0.658452,-0.941998,1.220276,0.126910,1.215960,-0.031406,1.369763,-0.429005,-0.995603,2.213520,...,1.141488,-0.499428,-0.907098,-2.310799,1.134422,-0.157661,0.250660,0.248243,-0.193869,-132.149886
2,-0.524154,0.748664,0.911736,1.270206,-1.347053,1.441366,-0.904817,0.549289,-0.112964,-0.542748,...,1.597874,-0.700074,0.160976,1.065414,0.754435,0.349558,-0.375362,0.265859,0.280804,60.219350
3,0.095497,-0.560597,0.606521,0.315178,-0.464615,-0.693220,-0.298582,0.436818,0.212657,1.168977,...,0.182767,0.624826,-1.167907,-1.238691,1.024234,-0.482477,0.545776,-1.937748,-1.472297,-374.732720
4,-1.160191,-0.096265,-1.656794,0.784624,0.015642,-0.013125,-0.250873,0.412087,-0.550389,-1.467188,...,-1.350385,-0.144729,-0.937136,-0.277679,0.579829,1.048830,0.491816,-0.985901,-0.562375,-79.070875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.377881,-1.052292,0.681293,-2.712770,0.969815,0.490391,-2.799748,-0.008237,0.393239,1.250895,...,-0.435035,-2.397281,-0.321910,0.753783,0.021555,2.125836,0.088730,-0.297962,0.147169,93.721048
14996,-1.130753,-0.298664,0.404196,0.193609,-0.903787,0.034225,-0.740811,-0.049095,0.527936,0.420968,...,0.941157,0.509872,0.399278,-1.474479,-1.169842,1.008214,0.861208,-1.448401,-0.802304,-30.982708
14997,-0.190190,0.038207,1.401782,0.544128,-0.198051,-1.003762,-0.216888,0.467130,-1.225987,-0.603110,...,-1.481234,-1.636574,-3.065831,-2.574360,1.366525,1.003513,2.025767,0.789690,0.542271,-211.186369
14998,-0.305903,0.043371,0.581206,-1.892132,0.805649,1.828304,-0.003037,0.598356,0.392362,-0.973656,...,1.334144,1.694686,-0.201254,1.889447,-0.519157,0.404197,0.918249,0.710569,-1.344158,-180.337519


In [4]:
# we create a new column called kfold and fill it with -1 
df["kfold"] = -1 

In [5]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target,kfold
0,0.140460,0.198421,-0.574314,1.060653,-0.758788,1.317201,0.877023,-0.549061,-0.546976,-2.148835,...,-0.219189,-0.109742,0.592466,-1.906363,0.013963,-0.315500,-0.265236,-0.243782,161.886006,-1
1,-0.658452,-0.941998,1.220276,0.126910,1.215960,-0.031406,1.369763,-0.429005,-0.995603,2.213520,...,-0.499428,-0.907098,-2.310799,1.134422,-0.157661,0.250660,0.248243,-0.193869,-132.149886,-1
2,-0.524154,0.748664,0.911736,1.270206,-1.347053,1.441366,-0.904817,0.549289,-0.112964,-0.542748,...,-0.700074,0.160976,1.065414,0.754435,0.349558,-0.375362,0.265859,0.280804,60.219350,-1
3,0.095497,-0.560597,0.606521,0.315178,-0.464615,-0.693220,-0.298582,0.436818,0.212657,1.168977,...,0.624826,-1.167907,-1.238691,1.024234,-0.482477,0.545776,-1.937748,-1.472297,-374.732720,-1
4,-1.160191,-0.096265,-1.656794,0.784624,0.015642,-0.013125,-0.250873,0.412087,-0.550389,-1.467188,...,-0.144729,-0.937136,-0.277679,0.579829,1.048830,0.491816,-0.985901,-0.562375,-79.070875,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.377881,-1.052292,0.681293,-2.712770,0.969815,0.490391,-2.799748,-0.008237,0.393239,1.250895,...,-2.397281,-0.321910,0.753783,0.021555,2.125836,0.088730,-0.297962,0.147169,93.721048,-1
14996,-1.130753,-0.298664,0.404196,0.193609,-0.903787,0.034225,-0.740811,-0.049095,0.527936,0.420968,...,0.509872,0.399278,-1.474479,-1.169842,1.008214,0.861208,-1.448401,-0.802304,-30.982708,-1
14997,-0.190190,0.038207,1.401782,0.544128,-0.198051,-1.003762,-0.216888,0.467130,-1.225987,-0.603110,...,-1.636574,-3.065831,-2.574360,1.366525,1.003513,2.025767,0.789690,0.542271,-211.186369,-1
14998,-0.305903,0.043371,0.581206,-1.892132,0.805649,1.828304,-0.003037,0.598356,0.392362,-0.973656,...,1.694686,-0.201254,1.889447,-0.519157,0.404197,0.918249,0.710569,-1.344158,-180.337519,-1


In [6]:
# the next step is to randomize the rows of the data 
df = df.sample(frac=1).reset_index(drop=True) 

In [7]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_92,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target,kfold
0,4.484661,0.623409,0.280025,0.656302,-1.142731,-0.530677,-1.112312,-1.422327,1.419715,1.700172,...,-1.588412,0.384667,-0.487902,-0.357230,0.119460,0.677076,-2.367212,-0.821370,48.346911,-1
1,0.451877,-1.446141,0.694440,0.539603,0.464911,-0.750645,0.170954,-0.284000,-0.588204,-0.344379,...,-0.169139,-0.409122,1.135580,1.905083,1.241423,-0.380315,1.350614,-0.269601,149.820816,-1
2,-0.624440,0.052349,-1.756215,0.190472,-1.472257,-0.872060,-0.291303,-0.006503,0.495774,-0.365020,...,-0.991750,0.939765,-0.030803,0.095336,0.408011,0.125801,-2.044243,0.397085,178.391101,-1
3,-0.427258,-0.767855,0.959233,-0.011064,-0.437715,0.079610,-0.356931,0.334606,0.688847,-1.223974,...,-0.581655,0.131470,-0.242292,-0.511431,-0.058561,1.794978,-0.250641,0.413724,-62.470241,-1
4,0.494302,-1.120785,-0.017868,-1.229976,-0.080626,-0.269022,-2.269166,0.163210,-0.048605,-0.019929,...,-1.034690,-0.714178,-1.971676,-0.801681,-0.112179,-1.090077,-0.520187,1.355187,-116.066688,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-1.655863,-2.442815,-0.203115,2.108243,0.052520,0.585678,-2.405433,-0.074645,-0.422706,-1.843877,...,0.156006,-1.771645,0.451973,0.543100,-0.538236,0.200344,-0.048972,0.028576,-194.820107,-1
14996,-0.600268,1.095462,-0.511658,0.384420,-0.869394,-0.233261,0.804399,-1.280111,-1.918195,-0.776111,...,0.599426,1.455737,0.456316,1.049237,0.127363,0.669815,2.610051,0.349601,-225.580685,-1
14997,1.269581,-1.014912,-0.353095,0.838359,-0.607913,-0.616725,0.959470,0.471328,0.988604,0.176276,...,0.716116,0.295293,-0.696693,-0.715848,-0.698772,1.923533,1.323805,0.025342,122.988359,-1
14998,2.014176,0.494406,0.223663,0.403644,-1.297151,0.130589,0.923919,-1.330848,0.957738,0.798746,...,0.921968,0.643023,-0.542506,0.259581,1.130302,-0.687388,-0.196707,0.094729,-232.657713,-1


In [8]:
# calculate the number of bins by Sturge's rule 
# I take the floor of the value, you can also 
# just round it 
num_bins = int(np.floor(1 + np.log2(len(df))))

In [9]:
num_bins

14

In [10]:
# bin targets
# https://note.nkmk.me/python-pandas-cut-qcut-binning/
df.loc[:, "bins"] = pd.cut(df["target"], bins=num_bins, labels=False) 

In [11]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target,kfold,bins
0,4.484661,0.623409,0.280025,0.656302,-1.142731,-0.530677,-1.112312,-1.422327,1.419715,1.700172,...,0.384667,-0.487902,-0.357230,0.119460,0.677076,-2.367212,-0.821370,48.346911,-1,8
1,0.451877,-1.446141,0.694440,0.539603,0.464911,-0.750645,0.170954,-0.284000,-0.588204,-0.344379,...,-0.409122,1.135580,1.905083,1.241423,-0.380315,1.350614,-0.269601,149.820816,-1,9
2,-0.624440,0.052349,-1.756215,0.190472,-1.472257,-0.872060,-0.291303,-0.006503,0.495774,-0.365020,...,0.939765,-0.030803,0.095336,0.408011,0.125801,-2.044243,0.397085,178.391101,-1,9
3,-0.427258,-0.767855,0.959233,-0.011064,-0.437715,0.079610,-0.356931,0.334606,0.688847,-1.223974,...,0.131470,-0.242292,-0.511431,-0.058561,1.794978,-0.250641,0.413724,-62.470241,-1,7
4,0.494302,-1.120785,-0.017868,-1.229976,-0.080626,-0.269022,-2.269166,0.163210,-0.048605,-0.019929,...,-0.714178,-1.971676,-0.801681,-0.112179,-1.090077,-0.520187,1.355187,-116.066688,-1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-1.655863,-2.442815,-0.203115,2.108243,0.052520,0.585678,-2.405433,-0.074645,-0.422706,-1.843877,...,-1.771645,0.451973,0.543100,-0.538236,0.200344,-0.048972,0.028576,-194.820107,-1,6
14996,-0.600268,1.095462,-0.511658,0.384420,-0.869394,-0.233261,0.804399,-1.280111,-1.918195,-0.776111,...,1.455737,0.456316,1.049237,0.127363,0.669815,2.610051,0.349601,-225.580685,-1,5
14997,1.269581,-1.014912,-0.353095,0.838359,-0.607913,-0.616725,0.959470,0.471328,0.988604,0.176276,...,0.295293,-0.696693,-0.715848,-0.698772,1.923533,1.323805,0.025342,122.988359,-1,8
14998,2.014176,0.494406,0.223663,0.403644,-1.297151,0.130589,0.923919,-1.330848,0.957738,0.798746,...,0.643023,-0.542506,0.259581,1.130302,-0.687388,-0.196707,0.094729,-232.657713,-1,5


In [12]:
# initiate the kfold class from model_selection module 
kf = model_selection.StratifiedKFold(n_splits=5) 

In [13]:
# fill the new kfold column 
# note that, instead of targets, we use bins! 
for f, (t_, v_) in enumerate(kf.split(X=df, y=df.bins.values)): 
    df.loc[v_, 'kfold'] = f 

C:\Users\iiats\Anaconda3\envs\Python36\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [14]:
df

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_93,f_94,f_95,f_96,f_97,f_98,f_99,target,kfold,bins
0,4.484661,0.623409,0.280025,0.656302,-1.142731,-0.530677,-1.112312,-1.422327,1.419715,1.700172,...,0.384667,-0.487902,-0.357230,0.119460,0.677076,-2.367212,-0.821370,48.346911,0,8
1,0.451877,-1.446141,0.694440,0.539603,0.464911,-0.750645,0.170954,-0.284000,-0.588204,-0.344379,...,-0.409122,1.135580,1.905083,1.241423,-0.380315,1.350614,-0.269601,149.820816,0,9
2,-0.624440,0.052349,-1.756215,0.190472,-1.472257,-0.872060,-0.291303,-0.006503,0.495774,-0.365020,...,0.939765,-0.030803,0.095336,0.408011,0.125801,-2.044243,0.397085,178.391101,0,9
3,-0.427258,-0.767855,0.959233,-0.011064,-0.437715,0.079610,-0.356931,0.334606,0.688847,-1.223974,...,0.131470,-0.242292,-0.511431,-0.058561,1.794978,-0.250641,0.413724,-62.470241,0,7
4,0.494302,-1.120785,-0.017868,-1.229976,-0.080626,-0.269022,-2.269166,0.163210,-0.048605,-0.019929,...,-0.714178,-1.971676,-0.801681,-0.112179,-1.090077,-0.520187,1.355187,-116.066688,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-1.655863,-2.442815,-0.203115,2.108243,0.052520,0.585678,-2.405433,-0.074645,-0.422706,-1.843877,...,-1.771645,0.451973,0.543100,-0.538236,0.200344,-0.048972,0.028576,-194.820107,4,6
14996,-0.600268,1.095462,-0.511658,0.384420,-0.869394,-0.233261,0.804399,-1.280111,-1.918195,-0.776111,...,1.455737,0.456316,1.049237,0.127363,0.669815,2.610051,0.349601,-225.580685,4,5
14997,1.269581,-1.014912,-0.353095,0.838359,-0.607913,-0.616725,0.959470,0.471328,0.988604,0.176276,...,0.295293,-0.696693,-0.715848,-0.698772,1.923533,1.323805,0.025342,122.988359,4,8
14998,2.014176,0.494406,0.223663,0.403644,-1.297151,0.130589,0.923919,-1.330848,0.957738,0.798746,...,0.643023,-0.542506,0.259581,1.130302,-0.687388,-0.196707,0.094729,-232.657713,4,5


In [15]:
# df.to_csv('./data/df1.csv')